In [1]:
# !pip3 install keras

In [2]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

import numpy as np

from keras import backend as K
import tensorflow as tf
B = tf.keras.backend

#Set seeds for reproducability

import os
os.environ['PYTHONHASHSEED'] = '0'
seed = 6
import random as rn
rn.seed(seed)
import numpy as np
np.random.seed(seed)
tf.random.set_seed(seed)

@tf.function(experimental_relax_shapes=True)
def gelu(x):
    return 0.5 * x * (1 + B.tanh(x * 0.7978845608 * (1 + 0.044715 * x * x)))


In [3]:
# Load Data

(x_train_init, y_train_init), (x_test_init, y_test_init) = cifar10.load_data()

In [4]:
# Data Prep

mean = np.mean(x_train_init,axis=(0,1,2,3))
std = np.std(x_train_init,axis=(0,1,2,3))
x_train = (x_train_init-mean)/(std)
x_test = (x_test_init-mean)/(std)

num_classes = 10
y_train = np_utils.to_categorical(y_train_init,num_classes)
y_test = np_utils.to_categorical(y_test_init,num_classes)

In [5]:
# Model Architecture

model=Sequential()
model.add(Conv2D(filters=32,kernel_size=(3,3),activation=gelu,input_shape=x_train.shape[1:], padding='same'))
model.add(Conv2D(filters=32,kernel_size=(3,3),activation=gelu, padding='same'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64,kernel_size=(3,3),activation=gelu, padding='same'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(filters=64,kernel_size=(3,3),activation=gelu, padding='same'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))


model.add(Flatten())
model.add(Dense(units=512, activation=gelu))
model.add(Dropout(0.4))
model.add(Dense(units=10,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(lr=0.003),metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 64)          3

In [6]:
def lr_scheduler(epoch):
    lrate = init_lr
    if epoch > 9:
        lrate = 0.0025
    if epoch > 19:
        lrate = 0.0015
    if epoch > 29:
        lrate = 0.0075
    if epoch > 44:
        lrate = 0.0035
    return lrate

def lr_decay(epoch):
    return init_lr * decay ** epoch

class LrHistory(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        print("Learning rate:", K.get_value(model.optimizer.lr))

        
early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=5, restore_best_weights=True),
# checkpoint = ModelCheckpoint("best_model_checkpoint.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'),
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, mode='min')

In [7]:
init_lr = 0.0035
decay = 0.9

history = model.fit(
    x_train,y_train,
    validation_data=(x_test, y_test),
    batch_size=32,
    callbacks=[
               keras.callbacks.LearningRateScheduler(lr_decay),
               LrHistory(),
               early_stop],
    epochs=100,
    shuffle=True
    )

Learning rate: 0.0035
Epoch 1/100
1563/1563 [==============================] - 20s 13ms/step - loss: 1.3895 - accuracy: 0.5100 - val_loss: 1.0523 - val_accuracy: 0.6317 - lr: 0.0035
Learning rate: 0.00315
Epoch 2/100
1563/1563 [==============================] - 20s 13ms/step - loss: 1.1873 - accuracy: 0.5928 - val_loss: 0.9987 - val_accuracy: 0.6577 - lr: 0.0032
Learning rate: 0.002835
Epoch 3/100
1563/1563 [==============================] - 21s 13ms/step - loss: 1.0900 - accuracy: 0.6269 - val_loss: 0.8998 - val_accuracy: 0.6945 - lr: 0.0028
Learning rate: 0.0025515
Epoch 4/100
1563/1563 [==============================] - 22s 14ms/step - loss: 1.0269 - accuracy: 0.6506 - val_loss: 0.8836 - val_accuracy: 0.6921 - lr: 0.0026
Learning rate: 0.00229635
Epoch 5/100
1563/1563 [==============================] - 20s 13ms/step - loss: 0.9533 - accuracy: 0.6763 - val_loss: 0.7994 - val_accuracy: 0.7257 - lr: 0.0023
Learning rate: 0.002066715
Epoch 6/100
1563/1563 [==============================

In [9]:
from sklearn.metrics import confusion_matrix
Y_pred = model.predict(x_test)
y_test_non_category = [ np.argmax(t) for t in y_test ]
y_predict_non_category = [ np.argmax(t) for t in Y_pred ]

cnf_matrix = confusion_matrix(y_test_non_category, y_predict_non_category)
print(cnf_matrix)
cm = cnf_matrix
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print(cm)

[[856   7  21  17  11   1   5  10  32  40]
 [ 13 893   2   3   1   0   6   1   8  73]
 [ 52   2 709  53  61  37  53  23   4   6]
 [ 21   3  49 621  50 142  64  26  10  14]
 [ 18   2  39  46 788  18  38  41   7   3]
 [  9   1  40 145  33 709  22  36   3   2]
 [  5   3  28  33  19  13 891   5   2   1]
 [ 13   2  18  45  30  40   4 844   1   3]
 [ 54  18   8   9   4   4   2   3 880  18]
 [ 20  38   4   9   3   0   3   2  15 906]]
[[0.856 0.007 0.021 0.017 0.011 0.001 0.005 0.01  0.032 0.04 ]
 [0.013 0.893 0.002 0.003 0.001 0.    0.006 0.001 0.008 0.073]
 [0.052 0.002 0.709 0.053 0.061 0.037 0.053 0.023 0.004 0.006]
 [0.021 0.003 0.049 0.621 0.05  0.142 0.064 0.026 0.01  0.014]
 [0.018 0.002 0.039 0.046 0.788 0.018 0.038 0.041 0.007 0.003]
 [0.009 0.001 0.04  0.145 0.033 0.709 0.022 0.036 0.003 0.002]
 [0.005 0.003 0.028 0.033 0.019 0.013 0.891 0.005 0.002 0.001]
 [0.013 0.002 0.018 0.045 0.03  0.04  0.004 0.844 0.001 0.003]
 [0.054 0.018 0.008 0.009 0.004 0.004 0.002 0.003 0.88  0.018]
 [

In [10]:
# Test Model
scores = model.evaluate(x_test, y_test, batch_size=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

10000/10000 [==============================] - 43s 4ms/step - loss: 0.5964 - accuracy: 0.8097

Test result: 80.970 loss: 0.596
